In [1]:
def add(a: int, b: int = 0) -> int:
    return a + b

In [ ]:
add(12)

In [2]:
from inspect import signature

In [ ]:
sig = signature(add)

In [ ]:
sig

In [ ]:
p = list(sig.parameters.values())[0]

In [ ]:
p.annotation

In [3]:
import meander as web

In [4]:
def call(fn, **kwargs):
    args = []
    sig = signature(fn)
    for par in sig.parameters.values():
        if par.name not in kwargs:
            if par.default == par.empty:
                raise AttributeError(f"missing argument '{par.name}'")
            value = par.default
        else:
            value = kwargs[par.name]
        if par.annotation != par.empty:
            if par.annotation == int:
                value = web.param.types.integer(value)
            elif par.annotation == bool:
                value = web.param.types.boolean(value)
            elif par.annotation == str:
                value = str(value)
        args.append(value)
        
    return fn(*args)

In [ ]:
call(add, a="1")

In [ ]:
dir.__hash__

In [12]:
def get_params(fn):
    
    def get_type(par):
        if par.annotation != par.empty:
            if par.annotation == int:
                return web.param.types.integer
            if par.annotation == bool:
                return web.param.types.boolean
            if par.annotation == str:
                return str
            if isinstance(par.annotation, web.Param):
                return par.annotation
            return lambda x: x

    params = []

    sig = signature(fn)
    for par in sig.parameters.values():

        param = web.param.Param(get_type(par))
        if par.default != par.empty:
            param.default = par.default
        param.name = par.name
        
        params.append(param)
                
    return params

In [13]:
def call(fn, *args, **kwargs):
    
    params = get_params(fn)
    
    if len(args) > len(params):
         raise AttributeError(f"extra attributes: {args[len(params):]}")
            
    for value, param in zip(args, params):
        if param.name in kwargs:
            raise AttributeError(f"duplicate attribute: {param.name}")
        kwargs[param.name] = value
        
    # connection_id
    # kwarg_sink
    
    normal = []
    for param in params:
        if param.name not in kwargs:
            if param.is_required:
                raise AttributeError(f"required attribute: {param.name}")
            normal.append(param.default)
        else:
            try:
                value = kwargs[param.name]
                normal.append(param.type(value))
            except ValueError as err:
                raise ValueError(f"{param.name} is {err}")
                
    return fn(*normal)

In [15]:
call(add, 12, 4)

16

In [19]:
def echo(a: int, b: str, con_id: str = ""):
    return dict(a=a, b=b)

In [21]:
echo(12)

TypeError: echo() missing 1 required positional argument: 'b'